In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np

In [4]:
# Set up Selenium WebDriver
driver_path = r"C:\Users\vempa\Downloads\chromedriver-win64 (1)\chromedriver-win64\chromedriver.exe"
service = Service(driver_path)
chrome_options = Options()
driver = webdriver.Chrome(service=service, options=chrome_options)

# Navigate to Zomato main page
zomato_URL = r"https://www.zomato.com/india"
driver.get(zomato_URL)

# Collect links from the main page
links = []
class_names = ['.sc-bke1zw-1.bcVYKA a', '.sc-bke1zw-1.MwaDg a', '.sc-bke1zw-1.fTtEw a']
for class_name in class_names:
    elements = driver.find_elements(By.CSS_SELECTOR, class_name)
    for elem in elements:
        href = elem.get_attribute("href")
        if href:
            links.append(href)

# driver.close()
print(f"Collected {len(links)} links.")

driver.close()

Collected 85 links.


In [15]:
driver = webdriver.Chrome(service=service, options=chrome_options)
l = []
names=[]
total={}
# driver = webdriver.Chrome()
for no,i in enumerate(links):
    l = []
    driver.get(i)
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")  # Scroll to load dynamic content
    time.sleep(5)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(8)  # Scroll to load dynamic content
    driver.execute_script("window.scrollTo(document.body.scrollHeight,0)")
    for j in driver.find_elements(By.XPATH, ".//div[@class='sc-evWYkj cRThYq']"):
        l.append(j.text)
    print(i,l)
    # for j in driver.find_elements(By.XPATH,".//div[@class='jumbo-tracker']"):
    #     l.append(j.text)
    for name in driver.find_elements(By.XPATH,'//*[@id="root"]/div/div/h1'): #//*[@id="root"]/div/div[8]/h1
        names.append(name.text)
        total[name.text] = l
    print(no,"-",i)
    driver.back()
driver.close()

print('length of names',len(names))
print('len of data',len(total))
# import numpy as np
df=pd.DataFrame()
restaurant = []
rating_ = []
food = []
cost = []
loc = []
dist = []
city=[]
raw=[]
max=5
for i,j in total.items():
    for k in j:
        d=k.split('\n')
        if 'Promoted' in d:
            d.pop(0)
        for f in d[:2]:
            if 'Flat' in f:
                d.remove(f)

        if len(d)==max:# or len(i.split('\n'))==max:
            # max=len(i.split('\n'))
            print(d)
        city.append(i)
        restaurant.append(d[0])
        rating_.append(d[1])
        food.append(d[2])
        cost.append(d[3])
        raw.append(str(k.replace('\n','-->')))
        try:
            dist.append(d[5])
            loc.append(d[4])
        except:
            dist.append(d[4])
            loc.append(np.nan)
df['city_name']=city
df['hotel_name']=restaurant 
df['food_type'] = food
df['Avg_cost']=cost
df['Rating']=rating_
df['Location']=loc
df['distance']=dist
df['raw_data']=raw

https://www.zomato.com/agra ['Promoted\nFlat 10% OFF\nQairo\n4.5\nNorth Indian, Italian, Asian, Continental, Biryani, Chinese, Salad, Desserts\n₹1,200 for two\nTajganj, Agra\n2.5 km', 'Promoted\nFlat 15% OFF\nThe Salt Cafe Kitchen & Bar\n4.9\nNorth Indian, Continental, Chinese, Italian, Finger Food\n₹1,800 for two\nTajganj, Agra\n3.8 km', 'Promoted\nFlat 15% OFF\nUnplugged Courtyard\n4.4\nMediterranean, North Indian, Continental, Chinese, Mexican, Thai, Italian, Coffee\n₹1,600 for two\nTajganj, Agra\n4.8 km', 'Promoted\nFlat 10% OFF\nDowntown Cafe\n4.2\nNorth Indian, Mughlai, Continental, Chinese, Beverages, Desserts\n₹1,600 for two\nRakabganj, Agra\n1.4 km', 'Promoted\nFlat 10% OFF\nHotel Moti Palace - Pankhuri Restaurant\n4.1\nNorth Indian, Chinese, Continental, South Indian, Desserts, Beverages\n₹1,000 for two\nRakabganj, Agra\n1.4 km', 'Promoted\nFlat 10% OFF\nBeep\n4.2\nNorth Indian, Chinese, Continental, Pizza, Pasta\n₹1,200 for two\nRakabganj, Agra\n2.5 km', 'Promoted\nFlat 15% 

In [16]:
df['city_name'].nunique()

79

In [17]:
df

city_name                               hotel_name  \
0            Best Food in Agra                                    Qairo   
1            Best Food in Agra              The Salt Cafe Kitchen & Bar   
2            Best Food in Agra                      Unplugged Courtyard   
3            Best Food in Agra                            Downtown Cafe   
4            Best Food in Agra  Hotel Moti Palace - Pankhuri Restaurant   
...                        ...                                      ...   
10987  Best Food in Vijayawada                             Tandoori Hub   
10988  Best Food in Vijayawada                               Puri Vurii   
10989  Best Food in Vijayawada                Aithey Biryani Restaurant   
10990  Best Food in Vijayawada              Alpaharam's Kitchen Secrets   
10991  Best Food in Vijayawada                        Sandwich & Shakes   

                                               food_type        Avg_cost  \
0      North Indian, Italian, Asian, Continental, Bir...  ₹1,200 for two   
1      North Indian, Continental, Chinese, Italian, F...  ₹1,800 for two   
2      Mediterranean, North Indian, Continental, Chin...  ₹1,600 for two   
3      North Indian, Mughlai, Continental, Chinese, B...  ₹1,600 for two   
4      North Indian, Chinese, Continental, South Indi...  ₹1,000 for two   
...                                                  ...             ...   
10987     Biryani, North Indian, Chinese, Seafood, Kebab    ₹400 for two   
10988  Street Food, Maharashtrian, North Indian, Raja...    ₹400 for two   
10989                                            Biryani    ₹300 for two   
10990  Biryani, North Indian, Fast Food, South Indian...    ₹500 for two   
10991  Pizza, Sandwich, Fast Food, Beverages, Dessert...    ₹400 for two   

      Rating                  Location              distance  \
0        4.5             Tajganj, Agra                2.5 km   
1        4.9             Tajganj, Agra                3.8 km   
2        4.4             Tajganj, Agra                4.8 km   
3        4.2           Rakabganj, Agra                1.4 km   
4        4.1           Rakabganj, Agra                1.4 km   
...      ...                       ...                   ...   
10987    4.1    Auto Nagar, Vijayawada                1.7 km   
10988    4.1      Labbipet, Vijayawada                 577 m   
10989    4.3  Gandhi Nagar, Vijayawada  Closes in 42 minutes   
10990    3.8   Governorpet, Vijayawada                2.3 km   
10991    4.0      Labbipet, Vijayawada                 612 m   

                                                raw_data  
0      Promoted\nFlat 10% OFF\nQairo\n4.5\nNorth Indi...  
1      Promoted\nFlat 15% OFF\nThe Salt Cafe Kitchen ...  
2      Promoted\nFlat 15% OFF\nUnplugged Courtyard\n4...  
3      Promoted\nFlat 10% OFF\nDowntown Cafe\n4.2\nNo...  
4      Promoted\nFlat 10% OFF\nHotel Moti Palace - Pa...  
...                                                  ...  
10987  Tandoori Hub\n4.1\nBiryani, North Indian, Chin...  
10988  Puri Vurii\n4.1\nStreet Food, Maharashtrian, N...  
10989  Aithey Biryani Restaurant\n4.3\nBiryani\n₹300 ...  
10990  Alpaharam's Kitchen Secrets\n3.8\nBiryani, Nor...  
10991  Sandwich & Shakes\n4.0\nPizza, Sandwich, Fast ...  

[10992 rows x 8 columns]

In [18]:
df.to_csv(r'C:\Users\vempa\OneDrive\Desktop\web scraping\zomato data_raw.csv')

In [2]:
data=pd.read_csv(r"C:\Users\vempa\OneDrive\Desktop\web scraping\zomato data_raw.csv")
# data.drop('Unnamed: 0',inplace=True,axis=0)
data.drop('Unnamed: 0',axis=1,inplace=True)
data

city_name                               hotel_name  \
0            Best Food in Agra                                    Qairo   
1            Best Food in Agra              The Salt Cafe Kitchen & Bar   
2            Best Food in Agra                      Unplugged Courtyard   
3            Best Food in Agra                            Downtown Cafe   
4            Best Food in Agra  Hotel Moti Palace - Pankhuri Restaurant   
...                        ...                                      ...   
10987  Best Food in Vijayawada                             Tandoori Hub   
10988  Best Food in Vijayawada                               Puri Vurii   
10989  Best Food in Vijayawada                Aithey Biryani Restaurant   
10990  Best Food in Vijayawada              Alpaharam's Kitchen Secrets   
10991  Best Food in Vijayawada                        Sandwich & Shakes   

                                               food_type        Avg_cost  \
0      North Indian, Italian, Asian, Continental, Bir...  ₹1,200 for two   
1      North Indian, Continental, Chinese, Italian, F...  ₹1,800 for two   
2      Mediterranean, North Indian, Continental, Chin...  ₹1,600 for two   
3      North Indian, Mughlai, Continental, Chinese, B...  ₹1,600 for two   
4      North Indian, Chinese, Continental, South Indi...  ₹1,000 for two   
...                                                  ...             ...   
10987     Biryani, North Indian, Chinese, Seafood, Kebab    ₹400 for two   
10988  Street Food, Maharashtrian, North Indian, Raja...    ₹400 for two   
10989                                            Biryani    ₹300 for two   
10990  Biryani, North Indian, Fast Food, South Indian...    ₹500 for two   
10991  Pizza, Sandwich, Fast Food, Beverages, Dessert...    ₹400 for two   

      Rating                  Location              distance  \
0        4.5             Tajganj, Agra                2.5 km   
1        4.9             Tajganj, Agra                3.8 km   
2        4.4             Tajganj, Agra                4.8 km   
3        4.2           Rakabganj, Agra                1.4 km   
4        4.1           Rakabganj, Agra                1.4 km   
...      ...                       ...                   ...   
10987    4.1    Auto Nagar, Vijayawada                1.7 km   
10988    4.1      Labbipet, Vijayawada                 577 m   
10989    4.3  Gandhi Nagar, Vijayawada  Closes in 42 minutes   
10990    3.8   Governorpet, Vijayawada                2.3 km   
10991    4.0      Labbipet, Vijayawada                 612 m   

                                                raw_data  
0      Promoted\nFlat 10% OFF\nQairo\n4.5\nNorth Indi...  
1      Promoted\nFlat 15% OFF\nThe Salt Cafe Kitchen ...  
2      Promoted\nFlat 15% OFF\nUnplugged Courtyard\n4...  
3      Promoted\nFlat 10% OFF\nDowntown Cafe\n4.2\nNo...  
4      Promoted\nFlat 10% OFF\nHotel Moti Palace - Pa...  
...                                                  ...  
10987  Tandoori Hub\n4.1\nBiryani, North Indian, Chin...  
10988  Puri Vurii\n4.1\nStreet Food, Maharashtrian, N...  
10989  Aithey Biryani Restaurant\n4.3\nBiryani\n₹300 ...  
10990  Alpaharam's Kitchen Secrets\n3.8\nBiryani, Nor...  
10991  Sandwich & Shakes\n4.0\nPizza, Sandwich, Fast ...  

[10992 rows x 8 columns]

In [4]:
new_distance=[]
for i in data['raw_data'].str.split('\n'):
    # print(i[-1])
    new_distance.append(i[-1])

In [5]:
data.insert(7,'new_distance',new_distance)

In [6]:
data

city_name                               hotel_name  \
0            Best Food in Agra                                    Qairo   
1            Best Food in Agra              The Salt Cafe Kitchen & Bar   
2            Best Food in Agra                      Unplugged Courtyard   
3            Best Food in Agra                            Downtown Cafe   
4            Best Food in Agra  Hotel Moti Palace - Pankhuri Restaurant   
...                        ...                                      ...   
10987  Best Food in Vijayawada                             Tandoori Hub   
10988  Best Food in Vijayawada                               Puri Vurii   
10989  Best Food in Vijayawada                Aithey Biryani Restaurant   
10990  Best Food in Vijayawada              Alpaharam's Kitchen Secrets   
10991  Best Food in Vijayawada                        Sandwich & Shakes   

                                               food_type        Avg_cost  \
0      North Indian, Italian, Asian, Continental, Bir...  ₹1,200 for two   
1      North Indian, Continental, Chinese, Italian, F...  ₹1,800 for two   
2      Mediterranean, North Indian, Continental, Chin...  ₹1,600 for two   
3      North Indian, Mughlai, Continental, Chinese, B...  ₹1,600 for two   
4      North Indian, Chinese, Continental, South Indi...  ₹1,000 for two   
...                                                  ...             ...   
10987     Biryani, North Indian, Chinese, Seafood, Kebab    ₹400 for two   
10988  Street Food, Maharashtrian, North Indian, Raja...    ₹400 for two   
10989                                            Biryani    ₹300 for two   
10990  Biryani, North Indian, Fast Food, South Indian...    ₹500 for two   
10991  Pizza, Sandwich, Fast Food, Beverages, Dessert...    ₹400 for two   

      Rating                  Location              distance new_distance  \
0        4.5             Tajganj, Agra                2.5 km       2.5 km   
1        4.9             Tajganj, Agra                3.8 km       3.8 km   
2        4.4             Tajganj, Agra                4.8 km       4.8 km   
3        4.2           Rakabganj, Agra                1.4 km       1.4 km   
4        4.1           Rakabganj, Agra                1.4 km       1.4 km   
...      ...                       ...                   ...          ...   
10987    4.1    Auto Nagar, Vijayawada                1.7 km       1.7 km   
10988    4.1      Labbipet, Vijayawada                 577 m        577 m   
10989    4.3  Gandhi Nagar, Vijayawada  Closes in 42 minutes       2.2 km   
10990    3.8   Governorpet, Vijayawada                2.3 km       2.3 km   
10991    4.0      Labbipet, Vijayawada                 612 m        612 m   

                                                raw_data  
0      Promoted\nFlat 10% OFF\nQairo\n4.5\nNorth Indi...  
1      Promoted\nFlat 15% OFF\nThe Salt Cafe Kitchen ...  
2      Promoted\nFlat 15% OFF\nUnplugged Courtyard\n4...  
3      Promoted\nFlat 10% OFF\nDowntown Cafe\n4.2\nNo...  
4      Promoted\nFlat 10% OFF\nHotel Moti Palace - Pa...  
...                                                  ...  
10987  Tandoori Hub\n4.1\nBiryani, North Indian, Chin...  
10988  Puri Vurii\n4.1\nStreet Food, Maharashtrian, N...  
10989  Aithey Biryani Restaurant\n4.3\nBiryani\n₹300 ...  
10990  Alpaharam's Kitchen Secrets\n3.8\nBiryani, Nor...  
10991  Sandwich & Shakes\n4.0\nPizza, Sandwich, Fast ...  

[10992 rows x 9 columns]

In [21]:
def change(d):
    if d['Location'].contains(r'for one', na=False, regex=True):
        print("yes")
data.apply(change,axis=1)

In [22]:
data.apply(change,axis=1)

AttributeError: 'str' object has no attribute 'contains'

In [13]:
data[data['Location'].str.contains(r'for one', na=False, regex=True)]

city_name hotel_name food_type  \
190   Best Food in Alappuzha    20% OFF       4.3   
207   Best Food in Alappuzha    ₹25 OFF       3.6   
218   Best Food in Alappuzha    20% OFF       3.8   
221   Best Food in Alappuzha   ₹100 OFF       3.4   
1757   Best Food in Junagadh    20% OFF       3.9   
...                      ...        ...       ...   
8256   Best Food in Jamnagar    30% OFF       4.2   
8264   Best Food in Jamnagar   ₹100 OFF       4.2   
8268   Best Food in Jamnagar   ₹100 OFF       3.9   
8270   Best Food in Jamnagar   ₹175 OFF       3.8   
8278   Best Food in Jamnagar   ₹150 OFF         -   

                                               Avg_cost  \
190        Kerala, Biryani, South Indian, Roast Chicken   
207                    North Indian, Chinese, Beverages   
218             Pizza, Fast Food, Burger, Shake, Juices   
221                           Bakery, Juices, Beverages   
1757  Sandwich, Pizza, Street Food, Fast Food, Bever...   
...                                                 ...   
8256  Cafe, Bakery, Tea, Shake, Beverages, Sandwich,...   
8264                                       South Indian   
8268     South Indian, North Indian, Chinese, Fast Food   
8270                                              Pizza   
8278                     North Indian, Biryani, Chinese   

                               Rating      Location distance new_distance  \
190                  Halal Food Court  ₹150 for one   24 min       24 min   
207                    Thai's Kitchen  ₹150 for one   35 min       35 min   
218                      Pizza Palace  ₹250 for one   32 min       32 min   
221                      Qissa Bakers  ₹100 for one   38 min       38 min   
1757              Jay Bhawani Vadapav  ₹100 for one   21 min       21 min   
...                               ...           ...      ...          ...   
8256                  Cafe Coffee Day  ₹300 for one   17 min       17 min   
8264                         Dosa Hub  ₹200 for one   21 min       21 min   
8268       Hotel Vishal International  ₹200 for one   27 min       27 min   
8270  The Italian day and night pizza  ₹200 for one   27 min       27 min   
8278               Panna Parcel Point  ₹200 for one   24 min       24 min   

                                               raw_data  
190   20% OFF\nHalal Food Court\n4.3\nKerala, Biryan...  
207   ₹25 OFF\nThai's Kitchen\n3.6\nNorth Indian, Ch...  
218   20% OFF\nPizza Palace\n3.8\nPizza, Fast Food, ...  
221   ₹100 OFF\nQissa Bakers\n3.4\nBakery, Juices, B...  
1757  20% OFF\nJay Bhawani Vadapav\n3.9\nSandwich, P...  
...                                                 ...  
8256  30% OFF\nCafe Coffee Day\n4.2\nCafe, Bakery, T...  
8264  ₹100 OFF\nDosa Hub\n4.2\nSouth Indian\n₹200 fo...  
8268  ₹100 OFF\nHotel Vishal International\n3.9\nSou...  
8270  ₹175 OFF\nThe Italian day and night pizza\n3.8...  
8278  ₹150 OFF\nPanna Parcel Point\n-\nNorth Indian,...  

[275 rows x 9 columns]

In [10]:
# data.to_csv(r'C:\Users\vempa\OneDrive\Desktop\web scraping\zomato data_raw_1.csv')

In [9]:
# pd.read_csv(r"C:\Users\vempa\OneDrive\Desktop\web scraping\zomato data_raw_1.csv")

In [53]:
def change(data):
    if '₹150 OFF' in data['hotel_name']:
        # data['hotel_name']= [i for i in data['raw_data'].str.split('\n')][1]
        print(data['raw_data'])


In [54]:
data.apply(change,axis=1)

₹150 OFF
NIC Ice Creams
4.6
Ice Cream, Desserts
₹100 for one
15 min
₹150 OFF
Noorjahan Hotel
3.8
South Indian, North Indian, Kerala
₹150 for one
33 min
₹150 OFF
Lulu Hyper Market
3.8
Kerala, North Indian, Pizza, Salad, Chinese, Street Food
₹200 for one
41 min
₹150 OFF
Dakshin
4.3
South Indian, North Indian, Chinese, Juices, Shake
₹150 for one
33 min
₹150 OFF
The Pizza Project By Oven Story
3.2
Pizza
₹100 for one
37 min
₹150 OFF
Grandnoor Xpress Dine
4.0
Arabian
₹200 for one
38 min
₹150 OFF
Pastel Bakes
3.6
Pizza, Burger, Desserts, Beverages, Shake
₹300 for one
41 min
₹150 OFF
Chillies Restaurant
4.4
Biryani, Seafood
₹150 for one
18 min
₹150 OFF
7th Heaven
4.5
Bakery, Desserts
₹150 for one
17 min
₹150 OFF
Chillies Restaurant
4.4
Biryani, North Indian, Chinese
₹150 for one
21 min
₹150 OFF
NIC Ice Creams
4.6
Ice Cream, Desserts
₹100 for one
17 min
₹150 OFF
Aroma Coffee House
4.2
Biryani, Continental, Beverages, Shake
₹150 for one
35 min
₹150 OFF
Chillies Restaurant
4.4
North Indian, Birya

0        None
1        None
2        None
3        None
4        None
         ... 
10987    None
10988    None
10989    None
10990    None
10991    None
Length: 10992, dtype: object

In [44]:
data['hotel_name'].value_counts()

hotel_name
Domino's Pizza            42
KFC                       39
Pizza Hut                 36
Burger King               30
₹150 OFF                  29
                          ..
RR's Joint                 1
The Thickshake Factory     1
NR Tiffin Center           1
RR Durbar                  1
2ND Wife                   1
Name: count, Length: 9520, dtype: int64

In [19]:
# data_1=pd.read_csv(r"C:\Users\vempa\OneDrive\Desktop\web scraping\zomato data.csv")
# data_2=pd.read_csv(r"C:\Users\vempa\OneDrive\Desktop\web scraping\zomato data_1.csv")
# data_3=pd.read_csv(r"C:\Users\vempa\OneDrive\Desktop\web scraping\zomato data_2.csv")
# data_4=pd.read_csv(r"C:\Users\vempa\OneDrive\Desktop\web scraping\zomato data_3.csv")

In [20]:
# data_1

In [21]:
# data_2

In [22]:
# data_3

In [23]:
# data_4

In [24]:
# data=pd.concat([data_1,data_2,data_3,data_4])
# # data

In [25]:
# data.drop('Unnamed: 0',axis=1,inplace=True)

In [5]:
# pd.set_option('display.max_rows',None)

In [26]:
# data.drop_duplicates(inplace=True)

In [27]:
# data.sort_values(by=['city_name','hotel_name'],inplace=True)
# data

In [28]:
# # data.duplicated().sum()
# data['hotel_name'].duplicated().sum()